In [1]:
#stacking sur plusieurs model heterogènes 


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from numpy import std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

from sklearn.metrics import mean_squared_error

import lightgbm as lgbm
from fbprophet import Prophet

import matplotlib.pyplot as plt


In [2]:
from metric_calcul.WRMSSE import WRMSSEEvaluator

In [3]:
def get_lb_rank(score):
    """
    score de la LB depuis le fichier de la leaderboard fourni par kaggle
    """
    df_lb = pd.read_csv("/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/data/datasets_687702_1205782_m5-forecasting-accuracy-publicleaderboard-rank.csv")

    return (df_lb.Score <= score).sum() + 1

In [4]:
## data
df_train_full = pd.read_csv("/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/data/sales_train_evaluation.csv")

df_calendar = pd.read_csv("/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/data/calendar.csv")
df_prices = pd.read_csv("/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/data/sell_prices.csv")
df_sample_submission = pd.read_csv("/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/data/sample_submission.csv")
df_sample_submission["order"] = range(df_sample_submission.shape[0])

df_train = df_train_full.iloc[:, :-28]
df_valid = df_train_full.iloc[:, -28:]

In [5]:
evaluator = WRMSSEEvaluator(df_train, df_valid, df_calendar, df_prices)

In [470]:
#import des deux models à stack 
light_df = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/submission_1_all_data.csv')

In [471]:
prophet_df = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/submission_prophet_store.csv')
#prophet_df = prophet_df.drop('Unnamed: 0',axis=1)

In [472]:
light_hugo = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/Final-LGBM-StoreFinal.csv')

In [473]:
lin_reg = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/finalLr.csv')

In [474]:
catboost_df = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/submission_catboost.csv')

In [475]:
light_df.set_index('id',inplace=True)
light_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0.913235,0.827773,0.819731,0.795553,1.049908,1.189933,1.169151,0.985722,0.944810,0.950615,...,1.061990,1.313827,1.215845,0.953166,0.851623,0.837490,0.851408,1.019072,1.255848,1.241322
FOODS_1_001_CA_2_validation,0.953068,1.072191,0.958676,1.116954,1.153383,1.506220,1.364111,0.930627,0.916615,0.907662,...,1.135599,1.509597,1.478175,1.020680,0.964297,0.962438,0.963363,1.196510,1.645398,1.346751
FOODS_1_001_CA_3_validation,1.128027,1.075911,0.912882,0.886881,0.960791,1.125676,1.226875,1.061736,1.076449,0.935067,...,1.034299,1.497057,1.659610,1.141541,1.059402,0.902611,0.892723,0.966101,1.250934,1.292633
FOODS_1_001_CA_4_validation,0.398870,0.353666,0.351945,0.355953,0.418473,0.431137,0.497521,0.392819,0.413391,0.400593,...,0.438389,0.469421,0.476798,0.373327,0.364115,0.366365,0.373522,0.422287,0.467946,0.470816
FOODS_1_001_TX_1_validation,0.198043,0.192291,0.184541,0.186598,0.196933,0.199780,0.238182,0.525956,0.491990,0.477307,...,0.396754,0.378252,0.351230,0.287096,0.278369,0.274433,0.274933,0.298888,0.349390,0.329039


In [476]:
prophet_df.set_index('id',inplace=True)
prophet_df = prophet_df.reindex(index=light_df.index)

In [477]:
prophet_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0.844725,0.508323,0.848453,1.085265,1.067066,1.142291,1.050212,0.988762,0.561000,0.686795,...,0.426399,0.773188,0.920340,0.907603,0.401668,0.507858,0.621234,0.641549,0.823351,0.799759
FOODS_1_001_CA_2_validation,1.222304,1.285444,0.764100,0.707444,0.144746,0.676650,0.693274,0.516161,0.769625,0.395815,...,0.941683,1.651486,1.637470,1.373902,1.672382,1.528826,1.923100,1.733604,2.451670,2.483024
FOODS_1_001_CA_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.361427,0.000000,0.000000,0.051680,...,0.000000,0.780988,2.304872,1.001455,0.874208,0.866208,0.802186,1.247342,1.469065,2.134627
FOODS_1_001_CA_4_validation,0.693561,0.351822,0.612617,0.562230,0.514827,0.439177,0.310649,0.519141,0.217218,0.521867,...,0.000000,0.000000,0.000000,0.139941,0.000000,0.097061,0.070028,0.128835,0.234942,0.278270
FOODS_1_001_TX_1_validation,0.277361,0.464180,0.252109,0.256720,0.043841,0.218123,0.275541,0.574597,0.833038,0.607489,...,0.589594,0.653032,0.334709,0.197193,0.210885,0.005971,0.137598,0.002234,0.091778,0.000000


In [478]:
light_hugo.set_index('id',inplace=True)

In [479]:
lin_reg.set_index('id',inplace=True)

In [480]:
catboost_df.set_index('id',inplace=True)

In [481]:
light_hugo_pas_final = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/finalfinal2.csv')
light_hugo_pas_final.set_index('id',inplace=True)

In [482]:
cat_hugo_pas_final = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/submission/submission_catboost.csv')

cat_hugo_pas_final =  cat_hugo_pas_final.set_index('id')



In [483]:
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge
from sklearn.model_selection import train_test_split

In [484]:
light_df

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0.913235,0.827773,0.819731,0.795553,1.049908,1.189933,1.169151,0.985722,0.944810,0.950615,...,1.061990,1.313827,1.215845,0.953166,0.851623,0.837490,0.851408,1.019072,1.255848,1.241322
FOODS_1_001_CA_2_validation,0.953068,1.072191,0.958676,1.116954,1.153383,1.506220,1.364111,0.930627,0.916615,0.907662,...,1.135599,1.509597,1.478175,1.020680,0.964297,0.962438,0.963363,1.196510,1.645398,1.346751
FOODS_1_001_CA_3_validation,1.128027,1.075911,0.912882,0.886881,0.960791,1.125676,1.226875,1.061736,1.076449,0.935067,...,1.034299,1.497057,1.659610,1.141541,1.059402,0.902611,0.892723,0.966101,1.250934,1.292633
FOODS_1_001_CA_4_validation,0.398870,0.353666,0.351945,0.355953,0.418473,0.431137,0.497521,0.392819,0.413391,0.400593,...,0.438389,0.469421,0.476798,0.373327,0.364115,0.366365,0.373522,0.422287,0.467946,0.470816
FOODS_1_001_TX_1_validation,0.198043,0.192291,0.184541,0.186598,0.196933,0.199780,0.238182,0.525956,0.491990,0.477307,...,0.396754,0.378252,0.351230,0.287096,0.278369,0.274433,0.274933,0.298888,0.349390,0.329039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2_evaluation,0.238312,0.233083,0.265550,0.249625,0.317396,0.387206,0.346807,0.245170,0.237167,0.232450,...,0.301433,0.366154,0.350247,0.250735,0.221903,0.231386,0.222293,0.296379,0.367074,0.348167
HOUSEHOLD_2_516_TX_3_evaluation,0.165081,0.157491,0.166611,0.159202,0.194096,0.215518,0.175947,0.135479,0.119721,0.110006,...,0.165941,0.183702,0.176602,0.148990,0.136513,0.137050,0.138747,0.178362,0.201133,0.188340
HOUSEHOLD_2_516_WI_1_evaluation,0.081770,0.079970,0.082175,0.086339,0.102502,0.113213,0.098699,0.090278,0.084209,0.080257,...,0.128916,0.137416,0.129176,0.090620,0.085476,0.086004,0.088786,0.133809,0.143734,0.133436


In [485]:
light_df.reset_index(inplace=True)

In [486]:
melt_light = pd.melt(light_df,id_vars = 'id',
                  value_vars = [col for col in light_df.columns if col.startswith("F")],
                  var_name = "d",
                  value_name = "sales")

In [487]:
melt_light

,id,d,sales
0,FOODS_1_001_CA_1_validation,F1,0.913235
1,FOODS_1_001_CA_2_validation,F1,0.953068
2,FOODS_1_001_CA_3_validation,F1,1.128027
3,FOODS_1_001_CA_4_validation,F1,0.398870
4,FOODS_1_001_TX_1_validation,F1,0.198043
...,...,...,...
1707435,HOUSEHOLD_2_516_TX_2_evaluation,F28,0.348167
1707436,HOUSEHOLD_2_516_TX_3_evaluation,F28,0.188340
1707437,HOUSEHOLD_2_516_WI_1_evaluation,F28,0.133436
1707438,HOUSEHOLD_2_516_WI_2_evaluation,F28,0.097414


In [488]:
prophet_df.reset_index(inplace = True)
light_hugo.reset_index(inplace = True)
lin_reg.reset_index(inplace = True)
catboost_df.reset_index(inplace = True)


melt_prophet = pd.melt(prophet_df,id_vars = 'id',
                  value_vars = [col for col in prophet_df.columns if col.startswith("F")],
                  var_name = "d",
                  value_name = "sales")

melt_hugo = pd.melt(light_hugo,id_vars = 'id',
                  value_vars = [col for col in light_hugo.columns if col.startswith("F")],
                  var_name = "d",
                  value_name = "sales")

melt_lr = pd.melt(lin_reg,id_vars = 'id',
                  value_vars = [col for col in lin_reg.columns if col.startswith("F")],
                  var_name = "d",
                  value_name = "sales")

melt_catboost = pd.melt(catboost_df,id_vars = 'id',
                  value_vars = [col for col in catboost_df.columns if col.startswith("F")],
                  var_name = "d",
                  value_name = "sales")

In [489]:
df_all_predict = melt_light.merge(melt_prophet, on=["id","d"], how="inner")

In [490]:
df_all_predict = df_all_predict.rename(columns= {"sales_x":"light_pred","sales_y":"prophet_pred"})


In [491]:
df_all_predict = df_all_predict.merge(melt_hugo, on=["id","d"], how="inner")
df_all_predict = df_all_predict.merge(melt_lr, on=["id","d"], how="inner")
df_all_predict = df_all_predict.merge(melt_catboost, on=["id","d"], how="inner")

In [492]:
df_all_predict

,id,d,light_pred,prophet_pred,sales_x,sales_y,sales
0,FOODS_1_001_CA_1_validation,F1,0.913235,0.844725,1.134179,0.860294,0.459556
1,FOODS_1_001_CA_2_validation,F1,0.953068,1.222304,1.165394,0.753267,0.953073
2,FOODS_1_001_CA_3_validation,F1,1.128027,0.000000,1.113303,1.269769,1.492621
3,FOODS_1_001_CA_4_validation,F1,0.398870,0.693561,0.477037,0.786913,0.443711
4,FOODS_1_001_TX_1_validation,F1,0.198043,0.277361,0.202364,0.307507,0.281287
...,...,...,...,...,...,...,...
1707435,HOUSEHOLD_2_516_TX_2_evaluation,F28,0.348167,0.000716,0.267701,0.590129,0.683567
1707436,HOUSEHOLD_2_516_TX_3_evaluation,F28,0.188340,0.061665,0.300083,1.595422,1.745296
1707437,HOUSEHOLD_2_516_WI_1_evaluation,F28,0.133436,0.092576,0.149075,0.093372,0.032309
1707438,HOUSEHOLD_2_516_WI_2_evaluation,F28,0.097414,0.117645,0.112047,0.172706,0.027400


In [493]:
df_all_predict = df_all_predict.rename(columns= {"sales_x":"hugo_pred","sales_y": "lr_pred","sales":"catboost_pred"})
df_all_predict

,id,d,light_pred,prophet_pred,hugo_pred,lr_pred,catboost_pred
0,FOODS_1_001_CA_1_validation,F1,0.913235,0.844725,1.134179,0.860294,0.459556
1,FOODS_1_001_CA_2_validation,F1,0.953068,1.222304,1.165394,0.753267,0.953073
2,FOODS_1_001_CA_3_validation,F1,1.128027,0.000000,1.113303,1.269769,1.492621
3,FOODS_1_001_CA_4_validation,F1,0.398870,0.693561,0.477037,0.786913,0.443711
4,FOODS_1_001_TX_1_validation,F1,0.198043,0.277361,0.202364,0.307507,0.281287
...,...,...,...,...,...,...,...
1707435,HOUSEHOLD_2_516_TX_2_evaluation,F28,0.348167,0.000716,0.267701,0.590129,0.683567
1707436,HOUSEHOLD_2_516_TX_3_evaluation,F28,0.188340,0.061665,0.300083,1.595422,1.745296
1707437,HOUSEHOLD_2_516_WI_1_evaluation,F28,0.133436,0.092576,0.149075,0.093372,0.032309
1707438,HOUSEHOLD_2_516_WI_2_evaluation,F28,0.097414,0.117645,0.112047,0.172706,0.027400


In [494]:
validation = pd.read_csv('/Users/jerem/cours_esiee_paris/E4/kaggle_walmart/data/sales_train_evaluation.csv')

In [495]:
melt_validation = validation.iloc[:,-28:]

In [496]:
melt_validation.insert(0, 'id', validation['id'])

In [497]:
melt_validation.columns = ['id','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16',
                          'F17','F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28']

In [498]:
validation_set = melt_validation.copy()
validation_set["id"] = validation_set["id"].str.replace("evaluation$", "validation")
validation_set

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0,0,0,2,2,0,0,0,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_validation,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_validation,0,0,1,1,0,2,1,1,0,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_validation,1,3,0,1,2,1,0,2,1,...,1,1,1,4,6,0,1,1,1,0


In [499]:
validation_set = pd.melt(validation_set,id_vars = 'id',
                  value_vars = [col for col in light_hugo.columns if col.startswith("F")],
                  var_name = "d",
                  value_name = "sales")

In [500]:
validation_set

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,F1,0
1,HOBBIES_1_002_CA_1_validation,F1,0
2,HOBBIES_1_003_CA_1_validation,F1,0
3,HOBBIES_1_004_CA_1_validation,F1,0
4,HOBBIES_1_005_CA_1_validation,F1,1
...,...,...,...
853715,FOODS_3_823_WI_3_validation,F28,1
853716,FOODS_3_824_WI_3_validation,F28,0
853717,FOODS_3_825_WI_3_validation,F28,2
853718,FOODS_3_826_WI_3_validation,F28,0


In [501]:
df_all_predict = df_all_predict[~df_all_predict.id.str.contains("evaluation")]

In [502]:
a = df_all_predict.reset_index()

In [503]:
df_all_predict

,id,d,light_pred,prophet_pred,hugo_pred,lr_pred,catboost_pred
0,FOODS_1_001_CA_1_validation,F1,0.913235,0.844725,1.134179,0.860294,0.459556
1,FOODS_1_001_CA_2_validation,F1,0.953068,1.222304,1.165394,0.753267,0.953073
2,FOODS_1_001_CA_3_validation,F1,1.128027,0.000000,1.113303,1.269769,1.492621
3,FOODS_1_001_CA_4_validation,F1,0.398870,0.693561,0.477037,0.786913,0.443711
4,FOODS_1_001_TX_1_validation,F1,0.198043,0.277361,0.202364,0.307507,0.281287
...,...,...,...,...,...,...,...
1676945,HOUSEHOLD_2_516_TX_2_validation,F28,0.348167,0.300319,0.267701,0.590129,0.683567
1676946,HOUSEHOLD_2_516_TX_3_validation,F28,0.188340,0.089935,0.300083,1.595422,1.745296
1676947,HOUSEHOLD_2_516_WI_1_validation,F28,0.133436,0.000000,0.149075,0.093372,0.032309
1676948,HOUSEHOLD_2_516_WI_2_validation,F28,0.097414,0.079341,0.112047,0.172706,0.027400


In [504]:
df_validation = validation_set.copy()
df_main = df_all_predict.copy()


In [505]:
merge_into_one = df_main.merge(df_validation, on=['id','d'], how='inner')

In [506]:
merge_into_one

,id,d,light_pred,prophet_pred,hugo_pred,lr_pred,catboost_pred,sales
0,FOODS_1_001_CA_1_validation,F1,0.913235,0.844725,1.134179,0.860294,0.459556,2
1,FOODS_1_001_CA_2_validation,F1,0.953068,1.222304,1.165394,0.753267,0.953073,0
2,FOODS_1_001_CA_3_validation,F1,1.128027,0.000000,1.113303,1.269769,1.492621,1
3,FOODS_1_001_CA_4_validation,F1,0.398870,0.693561,0.477037,0.786913,0.443711,0
4,FOODS_1_001_TX_1_validation,F1,0.198043,0.277361,0.202364,0.307507,0.281287,0
...,...,...,...,...,...,...,...,...
853715,HOUSEHOLD_2_516_TX_2_validation,F28,0.348167,0.300319,0.267701,0.590129,0.683567,1
853716,HOUSEHOLD_2_516_TX_3_validation,F28,0.188340,0.089935,0.300083,1.595422,1.745296,2
853717,HOUSEHOLD_2_516_WI_1_validation,F28,0.133436,0.000000,0.149075,0.093372,0.032309,0
853718,HOUSEHOLD_2_516_WI_2_validation,F28,0.097414,0.079341,0.112047,0.172706,0.027400,0


In [509]:
#instantiation de la regression lineaire
data_lr = merge_into_one.set_index('id')

data_lr['light_pred'] = data_lr['light_pred'].astype('float32')
data_lr['prophet_pred'] = data_lr['prophet_pred'].astype('float32')
data_lr['hugo_pred'] = data_lr['hugo_pred'].astype('float32')
data_lr['lr_pred'] = data_lr['lr_pred'].astype('float32')
data_lr['catboost_pred'] = data_lr['catboost_pred'].astype('float32')
data_lr['sales'] = data_lr['sales'].astype('int16')

#data_lr = pd.get_dummies(data_lr)

In [510]:
data_lr = data_lr.drop('d', axis=1)
data_lr = data_lr.drop('lr_pred', axis=1)
#data_lr = data_lr.drop('light_pred', axis=1)
#data_lr = data_lr.drop('hugo_pred', axis=1)
data_lr

,light_pred,prophet_pred,hugo_pred,catboost_pred,sales
id,,,,,
FOODS_1_001_CA_1_validation,0.913235,0.844725,1.134179,0.459556,2
FOODS_1_001_CA_2_validation,0.953068,1.222304,1.165394,0.953073,0
FOODS_1_001_CA_3_validation,1.128027,0.000000,1.113302,1.492621,1
FOODS_1_001_CA_4_validation,0.398870,0.693561,0.477037,0.443711,0
FOODS_1_001_TX_1_validation,0.198043,0.277361,0.202364,0.281287,0
...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2_validation,0.348167,0.300319,0.267701,0.683567,1
HOUSEHOLD_2_516_TX_3_validation,0.188340,0.089935,0.300083,1.745296,2
HOUSEHOLD_2_516_WI_1_validation,0.133436,0.000000,0.149075,0.032309,0


In [546]:
%%time
X = data_lr.drop('sales',axis=1)
y = data_lr['sales']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)





CPU times: user 331 ms, sys: 13.4 ms, total: 344 ms
Wall time: 348 ms


In [550]:
from sklearn.ensemble import RandomForestRegressor

In [553]:
rf = RandomForestRegressor(max_depth= 4)

rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [584]:
rf_prediction = rf.predict(X)
len(rf_prediction)

853720

In [585]:
rf_prediction = pd.DataFrame(rf_prediction)

In [586]:
rf_prediction['id'] = data_lr.index.values
#rf_prediction = rf_prediction.set_index('id')
rf_prediction

,0,id
0,0.609149,FOODS_1_001_CA_1_validation
1,1.019282,FOODS_1_001_CA_2_validation
2,1.271439,FOODS_1_001_CA_3_validation
3,0.609149,FOODS_1_001_CA_4_validation
4,0.120585,FOODS_1_001_TX_1_validation
...,...,...
853715,0.609149,HOUSEHOLD_2_516_TX_2_validation
853716,1.642486,HOUSEHOLD_2_516_TX_3_validation
853717,0.120585,HOUSEHOLD_2_516_WI_1_validation
853718,0.120585,HOUSEHOLD_2_516_WI_2_validation


In [576]:
t = merge_into_one.copy()

In [581]:
t = t.loc[:,'id':'d']
t

,id,d
0,FOODS_1_001_CA_1_validation,F1
1,FOODS_1_001_CA_2_validation,F1
2,FOODS_1_001_CA_3_validation,F1
3,FOODS_1_001_CA_4_validation,F1
4,FOODS_1_001_TX_1_validation,F1
...,...,...
853715,HOUSEHOLD_2_516_TX_2_validation,F28
853716,HOUSEHOLD_2_516_TX_3_validation,F28
853717,HOUSEHOLD_2_516_WI_1_validation,F28
853718,HOUSEHOLD_2_516_WI_2_validation,F28


In [590]:
rf_prediction['d'] = t['d']
rf_prediction = rf_prediction.rename(columns={0:'sales'})
rf_prediction

,sales,id,d
0,0.609149,FOODS_1_001_CA_1_validation,F1
1,1.019282,FOODS_1_001_CA_2_validation,F1
2,1.271439,FOODS_1_001_CA_3_validation,F1
3,0.609149,FOODS_1_001_CA_4_validation,F1
4,0.120585,FOODS_1_001_TX_1_validation,F1
...,...,...,...
853715,0.609149,HOUSEHOLD_2_516_TX_2_validation,F28
853716,1.642486,HOUSEHOLD_2_516_TX_3_validation,F28
853717,0.120585,HOUSEHOLD_2_516_WI_1_validation,F28
853718,0.120585,HOUSEHOLD_2_516_WI_2_validation,F28


In [591]:
result = rf_prediction.copy()

In [592]:
result = result.pivot(index='id', columns='d',values= 'sales')

In [593]:
result

d,F1,F10,F11,F12,F13,F14,F15,F16,F17,F18,...,F26,F27,F28,F3,F4,F5,F6,F7,F8,F9
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,0.609149,1.278880,1.235421,1.642486,0.609149,0.609149,0.609149,0.609149,0.609149,0.609149,...,0.609149,0.120585,0.120585,0.349476,0.349476,0.609149,2.032108,2.032108,1.642486,1.242583
FOODS_1_001_CA_2_validation,1.019282,0.609149,0.609149,0.609149,0.626049,0.690624,0.609149,0.609149,0.609149,0.609149,...,0.603892,1.235421,1.235421,0.609149,0.620345,1.063773,0.609149,0.678066,0.609149,0.609149
FOODS_1_001_CA_3_validation,1.271439,0.446258,0.349476,0.609149,0.520306,0.120585,0.120585,0.120585,0.120585,0.120585,...,1.235421,2.032108,2.032108,1.235421,1.235421,1.278880,0.609149,0.609149,0.603892,0.446258
FOODS_1_001_CA_4_validation,0.609149,0.120585,0.120585,0.120585,0.192033,0.130063,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.446258,0.446258,0.609149,0.120585,0.120585,0.120585,0.120585
FOODS_1_001_TX_1_validation,0.120585,0.609149,0.609149,0.609149,1.145217,1.070767,0.609149,0.609149,0.609149,0.609149,...,1.642486,1.235421,1.235421,0.120585,0.120585,0.120585,0.609149,0.609149,0.609149,0.609149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.609149,0.609149,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
HOUSEHOLD_2_516_TX_3_validation,0.120585,0.451360,0.378211,0.609149,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,1.642486,1.642486,0.120585,0.120585,0.120585,0.609149,0.609149,0.541007,0.451360
HOUSEHOLD_2_516_WI_1_validation,0.120585,0.120585,0.120585,0.120585,0.192033,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585


In [602]:
d = result[['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16',
                          'F17','F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28']]
d = d.reset_index()
d

d,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.609149,0.349476,0.349476,0.349476,0.609149,2.032108,2.032108,1.642486,1.242583,...,0.609149,0.649215,0.672544,0.609149,0.609149,0.609149,0.609149,0.609149,0.120585,0.120585
1,FOODS_1_001_CA_2_validation,1.019282,0.609149,0.609149,0.620345,1.063773,0.609149,0.678066,0.609149,0.609149,...,0.609149,0.609149,0.609149,0.541007,0.349476,0.349476,0.520306,0.603892,1.235421,1.235421
2,FOODS_1_001_CA_3_validation,1.271439,1.235421,1.235421,1.235421,1.278880,0.609149,0.609149,0.603892,0.446258,...,0.120585,1.235421,1.642486,1.235421,1.131969,1.131969,1.131969,1.235421,2.032108,2.032108
3,FOODS_1_001_CA_4_validation,0.609149,0.446258,0.446258,0.446258,0.609149,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.206348,0.211102,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
4,FOODS_1_001_TX_1_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149,0.609149,0.609149,...,0.609149,2.032108,2.032108,1.642486,1.278880,1.278880,1.286169,1.642486,1.235421,1.235421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149
30486,HOUSEHOLD_2_516_TX_3_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149,0.541007,0.451360,...,0.120585,0.130063,0.211102,0.120585,0.120585,0.120585,0.120585,0.120585,1.642486,1.642486
30487,HOUSEHOLD_2_516_WI_1_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
30488,HOUSEHOLD_2_516_WI_2_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585


In [603]:
e = d.copy()
e["id"] = d["id"].str.replace("validation$", "evaluation")
e

d,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,0.609149,0.349476,0.349476,0.349476,0.609149,2.032108,2.032108,1.642486,1.242583,...,0.609149,0.649215,0.672544,0.609149,0.609149,0.609149,0.609149,0.609149,0.120585,0.120585
1,FOODS_1_001_CA_2_evaluation,1.019282,0.609149,0.609149,0.620345,1.063773,0.609149,0.678066,0.609149,0.609149,...,0.609149,0.609149,0.609149,0.541007,0.349476,0.349476,0.520306,0.603892,1.235421,1.235421
2,FOODS_1_001_CA_3_evaluation,1.271439,1.235421,1.235421,1.235421,1.278880,0.609149,0.609149,0.603892,0.446258,...,0.120585,1.235421,1.642486,1.235421,1.131969,1.131969,1.131969,1.235421,2.032108,2.032108
3,FOODS_1_001_CA_4_evaluation,0.609149,0.446258,0.446258,0.446258,0.609149,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.206348,0.211102,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
4,FOODS_1_001_TX_1_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149,0.609149,0.609149,...,0.609149,2.032108,2.032108,1.642486,1.278880,1.278880,1.286169,1.642486,1.235421,1.235421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149
30486,HOUSEHOLD_2_516_TX_3_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149,0.541007,0.451360,...,0.120585,0.130063,0.211102,0.120585,0.120585,0.120585,0.120585,0.120585,1.642486,1.642486
30487,HOUSEHOLD_2_516_WI_1_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585


In [605]:
d = pd.concat([d,e], axis=0,sort= False, ignore_index= True)
d

d,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.609149,0.349476,0.349476,0.349476,0.609149,2.032108,2.032108,1.642486,1.242583,...,0.609149,0.649215,0.672544,0.609149,0.609149,0.609149,0.609149,0.609149,0.120585,0.120585
1,FOODS_1_001_CA_2_validation,1.019282,0.609149,0.609149,0.620345,1.063773,0.609149,0.678066,0.609149,0.609149,...,0.609149,0.609149,0.609149,0.541007,0.349476,0.349476,0.520306,0.603892,1.235421,1.235421
2,FOODS_1_001_CA_3_validation,1.271439,1.235421,1.235421,1.235421,1.278880,0.609149,0.609149,0.603892,0.446258,...,0.120585,1.235421,1.642486,1.235421,1.131969,1.131969,1.131969,1.235421,2.032108,2.032108
3,FOODS_1_001_CA_4_validation,0.609149,0.446258,0.446258,0.446258,0.609149,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.206348,0.211102,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
4,FOODS_1_001_TX_1_validation,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149,0.609149,0.609149,...,0.609149,2.032108,2.032108,1.642486,1.278880,1.278880,1.286169,1.642486,1.235421,1.235421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_TX_2_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149
60976,HOUSEHOLD_2_516_TX_3_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.609149,0.609149,0.541007,0.451360,...,0.120585,0.130063,0.211102,0.120585,0.120585,0.120585,0.120585,0.120585,1.642486,1.642486
60977,HOUSEHOLD_2_516_WI_1_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585
60978,HOUSEHOLD_2_516_WI_2_evaluation,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,...,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585,0.120585


In [606]:
d.to_csv('../../rf_stacking.csv')

In [588]:
rf.score(X_test,y_test)

0.7711803493783203

In [562]:
a = rf.predict(X_test)
len(a)

256116

In [ ]:
#reg_lin = LinearRegression()
elastic = ElasticNet(alpha=1, l1_ratio= 0,)
#rid = Ridge(alpha=1.)

elastic.fit(X_train, y_train)

In [539]:
y_pred = rf.predict(X_test)

In [540]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [541]:
score = sqrt(mean_squared_error(y_test, y_pred))
score

1.7722212195850329

In [542]:
y_pred

array([3.9667108 , 0.22223744, 0.20882533, ..., 1.0373013 , 0.37381044,
       1.6850665 ], dtype=float32)

In [543]:
elastic.coef_

array([0.14553887, 0.06844389, 0.17360252, 0.582177  ], dtype=float32)

In [544]:
coefficients = pd.DataFrame({"Feature":X.columns,"Coefficients":np.transpose(rf.coef_)})

In [545]:
coefficients

,Feature,Coefficients
0,light_pred,0.145539
1,prophet_pred,0.068444
2,hugo_pred,0.173603
3,catboost_pred,0.582177
